Tujuan: memprediksi `case` dan `unit_cost` bulanan

Notebook ini digunakan untuk eksplorasi dan validasi untuk pengembangan model. Hasil prediksi dan model final yang digunakan terdapat di `train_predict.py`

In [1]:
import os
os.chdir('../')

In [2]:
import numpy as np
import pandas as pd
import lightgbm as lgb
import random
import warnings
warnings.filterwarnings('ignore')

from src.features import generate_date_features, get_cross_tkp_metrics, generate_lag_features
from src.util import seed_everything, mae, mape, read_data, create_folds
pd.set_option('display.max_columns', 200)

Let's create a utility functions first

In [3]:
# Default parameters
TARGET = ['case', 'unit_cost']
SEED = 2021
categorical_features = [
    'kddati2',
    'tkp',
    'id',
    'month',
    'year'
]
rs_features = [
    'a', 'b', 'c', 'cb', 'd', 'ds', 'gd', 'hd', 
    'i1', 'i2', 'i3', 'i4', 
    'kb', 'kc', 'kg', 'ki', 'kj', 'kk', 'kl', 'km', 'ko', 'kp', 'kt', 'ku', 
    's', 'sa', 'sb', 'sc', 'sd'
]
remove_features = ['tglpelayanan','row_id','cat','kfold']
numerical_features = ['peserta'] + TARGET + ['case2', 'unit_cost2']
FOLD = 5

## Data Preparation

In [18]:
# Preprocessing
df = read_data(dir='data')
df['id'] = df['kddati2'].astype(str) + '-' + df['tkp'].astype(str)
# df['rs_total'] = df[rs_features].sum(axis=1)

# Features engineering
df = generate_date_features(df)
df = get_cross_tkp_metrics(df)
df = generate_lag_features(df, features=numerical_features)

# Remove unused variables
df = df.drop(columns=['tglpelayanan_lag_1','tglpelayanan_lag_-1'])

# Create folds
df = create_folds(df, cv_split=5, seed=SEED)

66083 16521
66083 16521
66083 16521
66083 16521
66084 16520


In [5]:
df.head()

,row_id,tglpelayanan,kddati2,tkp,peserta,a,b,c,cb,d,ds,gd,hd,i1,i2,i3,i4,kb,kc,kg,ki,kj,kk,kl,km,ko,kp,kt,ku,s,sa,sb,sc,sd,case,unit_cost,cat,id,month,year,case2,unit_cost2,peserta_lag_1,case_lag_1,unit_cost_lag_1,case2_lag_1,unit_cost2_lag_1,peserta_lag_-1,case_lag_-1,unit_cost_lag_-1,case2_lag_-1,unit_cost2_lag_-1,ds_case2,ds_case_lag_1,ds_case2_lag_1,ds_case_lag_-1,ds_case2_lag_-1,kfold
0,20538,2014-01-01,1,30,76082,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,5557.0,215707.797553,Train,1-30,1,2014,1440.0,3.721830e+06,NaN,NaN,NaN,NaN,NaN,119291.0,6311.0,210940.329266,1763.0,4.049682e+06,0.0,NaN,NaN,-2.0,-3.0,4
1,13196,2014-03-01,1,30,119291,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,6311.0,210940.329266,Train,1-30,3,2014,1440.0,3.721830e+06,76082.0,5557.0,215707.797553,1440.0,3.721830e+06,121689.0,7295.0,210355.814256,1763.0,4.049682e+06,2.0,2.0,2.0,-1.0,-1.0,1
2,21066,2014-04-01,1,30,121689,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,7295.0,210355.814256,Train,1-30,4,2014,1763.0,4.049682e+06,119291.0,6311.0,210940.329266,1440.0,3.721830e+06,135725.0,6674.0,214941.752023,1543.0,4.378975e+06,0.0,1.0,3.0,-1.0,-1.0,4
3,42734,2014-05-01,1,30,135725,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,6674.0,214941.752023,Train,1-30,5,2014,1543.0,4.378975e+06,121689.0,7295.0,210355.814256,1763.0,4.049682e+06,133457.0,7068.0,210145.531551,1505.0,4.339726e+06,0.0,1.0,1.0,-1.0,-1.0,0
4,30314,2014-06-01,1,30,133457,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,7068.0,210145.531551,Train,1-30,6,2014,1505.0,4.339726e+06,135725.0,6674.0,214941.752023,1543.0,4.378975e+06,138340.0,6366.0,215640.201697,1364.0,4.175257e+06,0.0,1.0,1.0,-2.0,-1.0,0


## Modeling

### Naive Baseline

In [6]:
naive_baseline_df = df[['tglpelayanan','id','case','unit_cost','case_lag_1','case_lag_-1','unit_cost_lag_1','unit_cost_lag_-1','kfold','cat']].copy()
naive_baseline_df[['case_lag_1','unit_cost_lag_1']] = naive_baseline_df.groupby(['id'])[['case_lag_1','unit_cost_lag_1']].bfill()
naive_baseline_df[['case_lag_-1','unit_cost_lag_-1']] = naive_baseline_df[['case_lag_-1','unit_cost_lag_-1']].fillna(1)
naive_baseline_df = naive_baseline_df[naive_baseline_df['cat'] == 'Train']

In [7]:
for fold in range(FOLD):
    mape_case = mape(naive_baseline_df[naive_baseline_df['kfold'] == fold]['case'], 
                     naive_baseline_df[naive_baseline_df['kfold'] == fold]['case_lag_1'])
    mape_cost = mape(naive_baseline_df[naive_baseline_df['kfold'] == fold]['unit_cost'], 
                     naive_baseline_df[naive_baseline_df['kfold'] == fold]['unit_cost_lag_1'])

    print('Fold-{}: MAPE Case = {:.2f}%, MAPE Cost = {:.2f}%'.format(fold+1, mape_case, mape_cost))

mape_case = mape(naive_baseline_df['case'], 
                 naive_baseline_df['case_lag_1'])
mape_cost = mape(naive_baseline_df['unit_cost'], 
                 naive_baseline_df['unit_cost_lag_1'])
print('All: MAPE Case = {:.2f}%, MAPE Cost = {:.2f}%'.format(mape_case, mape_cost))

Fold-1: MAPE Case = 74.22%, MAPE Cost = 4.89%
Fold-2: MAPE Case = 47.78%, MAPE Cost = 5.20%
Fold-3: MAPE Case = 85.34%, MAPE Cost = 4.93%
Fold-4: MAPE Case = 250.23%, MAPE Cost = 4.89%
Fold-5: MAPE Case = 486.81%, MAPE Cost = 4.72%
All: MAPE Case = 189.00%, MAPE Cost = 4.92%


In [8]:
naive_baseline_df.loc[naive_baseline_df['tglpelayanan'] == '2021-06-01', 'case_lag_1'] = 1
for fold in range(FOLD):
    mape_case = mape(naive_baseline_df[naive_baseline_df['kfold'] == fold]['case'], 
                     naive_baseline_df[naive_baseline_df['kfold'] == fold]['case_lag_1'])
    mape_cost = mape(naive_baseline_df[naive_baseline_df['kfold'] == fold]['unit_cost'], 
                     naive_baseline_df[naive_baseline_df['kfold'] == fold]['unit_cost_lag_1'])

    print('Fold-{}: MAPE Case = {:.2f}%, MAPE Cost = {:.2f}%'.format(fold+1, mape_case, mape_cost))

mape_case = mape(naive_baseline_df['case'], 
                 naive_baseline_df['case_lag_1'])
mape_cost = mape(naive_baseline_df['unit_cost'], 
                 naive_baseline_df['unit_cost_lag_1'])
print('All: MAPE Case = {:.2f}%, MAPE Cost = {:.2f}%'.format(mape_case, mape_cost))

Fold-1: MAPE Case = 57.85%, MAPE Cost = 4.89%
Fold-2: MAPE Case = 46.45%, MAPE Cost = 5.20%
Fold-3: MAPE Case = 61.88%, MAPE Cost = 4.93%
Fold-4: MAPE Case = 70.61%, MAPE Cost = 4.89%
Fold-5: MAPE Case = 55.63%, MAPE Cost = 4.72%
All: MAPE Case = 58.50%, MAPE Cost = 4.92%


In [9]:
for fold in range(FOLD):
    mape_case = mape(naive_baseline_df[naive_baseline_df['kfold'] == fold]['case'], 
                     naive_baseline_df[naive_baseline_df['kfold'] == fold]['case_lag_-1'])
    mape_cost = mape(naive_baseline_df[naive_baseline_df['kfold'] == fold]['unit_cost'], 
                     naive_baseline_df[naive_baseline_df['kfold'] == fold]['unit_cost_lag_-1'])

    print('Fold-{}: MAPE Case = {:.2f}%, MAPE Cost = {:.2f}%'.format(fold+1, mape_case, mape_cost))

mape_case = mape(naive_baseline_df['case'], 
                 naive_baseline_df['case_lag_-1'])
mape_cost = mape(naive_baseline_df['unit_cost'], 
                 naive_baseline_df['unit_cost_lag_-1'])
print('All: MAPE Case = {:.2f}%, MAPE Cost = {:.2f}%'.format(mape_case, mape_cost))

Fold-1: MAPE Case = 28.07%, MAPE Cost = 6.66%
Fold-2: MAPE Case = 22.44%, MAPE Cost = 7.67%
Fold-3: MAPE Case = 32.37%, MAPE Cost = 6.89%
Fold-4: MAPE Case = 23.27%, MAPE Cost = 7.10%
Fold-5: MAPE Case = 38.27%, MAPE Cost = 6.70%
All: MAPE Case = 28.89%, MAPE Cost = 7.00%


In [23]:
for fold in range(FOLD):
    mape_case = mape(naive_baseline_df[naive_baseline_df['kfold'] == fold]['case'], 
                     (naive_baseline_df[naive_baseline_df['kfold'] == fold]['case_lag_-1'] + naive_baseline_df[naive_baseline_df['kfold'] == fold]['case_lag_1'])/2 )
    mape_cost = mape(naive_baseline_df[naive_baseline_df['kfold'] == fold]['unit_cost'], 
                     (naive_baseline_df[naive_baseline_df['kfold'] == fold]['unit_cost_lag_-1'] + naive_baseline_df[naive_baseline_df['kfold'] == fold]['unit_cost_lag_1'])/2)

    print('Fold-{}: MAPE Case = {:.2f}%, MAPE Cost = {:.2f}%'.format(fold+1, mape_case, mape_cost))

mape_case = mape(naive_baseline_df['case'], 
                 (naive_baseline_df[naive_baseline_df['kfold'] == fold]['case_lag_-1'] + naive_baseline_df[naive_baseline_df['kfold'] == fold]['case_lag_1'])/2)
mape_cost = mape(naive_baseline_df['unit_cost'], 
                 (naive_baseline_df[naive_baseline_df['kfold'] == fold]['unit_cost_lag_-1'] + naive_baseline_df[naive_baseline_df['kfold'] == fold]['unit_cost_lag_1'])/2)
print('All: MAPE Case = {:.2f}%, MAPE Cost = {:.2f}%'.format(mape_case, mape_cost))

Fold-1: MAPE Case = 39.55%, MAPE Cost = 4.78%
Fold-2: MAPE Case = 30.94%, MAPE Cost = 5.44%
Fold-3: MAPE Case = 43.73%, MAPE Cost = 4.93%
Fold-4: MAPE Case = 43.57%, MAPE Cost = 5.01%
Fold-5: MAPE Case = 43.58%, MAPE Cost = 4.76%
All: MAPE Case = 43.58%, MAPE Cost = 4.76%


### LightGBM

In [10]:
def process_train_lgb(
    df,
    target=TARGET[0],
    seed=SEED,
    verbose=500,
    validation=True,
    remove_additional_features=[],
    kfold=0,
    use_log=False
):
    df = df.copy()
    df_test = df[df['cat'] == 'Test']
    df = df[df['cat'] == 'Train']
    local_params = lgb_params.copy()

    if use_log:
        df[target] = np.log1p(df[target])

    # Categorical features
    for col in categorical_features:
        try:
            df[col] = df[col].astype('category')
            df_test[col] = df_test[col].astype('category')
        except:
            pass

    # All features
    remove_additional_features_selected = list(set(remove_additional_features) & set(df.columns.tolist()))  
    all_features = [col for col in list(df) if col not in (remove_features + remove_additional_features + TARGET)]

    print(all_features)

    # Split train and valid
    if validation:
        train_data = lgb.Dataset(df[df['kfold'] != kfold][all_features], label=df[df['kfold'] != kfold][target])
        valid_data = lgb.Dataset(df[df['kfold'] == kfold][all_features], label=df[df['kfold'] == kfold][target])
    else:
        train_data = lgb.Dataset(df[all_features], label=df[target])

    # Training process
    seed_everything()
    if validation:
        estimator = lgb.train(local_params,
                              train_data,
                              valid_sets = [train_data,valid_data],
                              verbose_eval = verbose)
        temp_df = df[df['kfold'] == kfold]

    else:
        if 'early_stopping_rounds' in local_params: 
            del local_params['early_stopping_rounds']
        estimator = lgb.train(local_params,
                              train_data,
                              valid_sets = [train_data],
                              verbose_eval = verbose)
        temp_df = df_test

    temp_df['predict_' + target] = estimator.predict(temp_df[all_features])

    if use_log:
        temp_df['predict_' + target] = np.expm1(temp_df['predict_' + target])
        temp_df[target] = np.expm1(temp_df[target])


    temp_df['predict_' + target] = temp_df['predict_' + target].clip(lower=1)

    if validation:
        print('MAPE CV-{} is {:.2f}%'.format(kfold+1, mape(temp_df[target], temp_df['predict_' + target])))
        print('MAE CV-{} is {:.2f}'.format(kfold+1, mae(temp_df[target], temp_df['predict_' + target])))

    temp_df = temp_df[['row_id','tglpelayanan','kddati2','tkp',target,'predict_' + target,'kfold']]

    return estimator, temp_df

In [19]:
df[['case_lag_1','unit_cost_lag_1']] = df.groupby(['id'])[['case_lag_1','unit_cost_lag_1']].bfill()
df[['case_lag_-1']] = df[['case_lag_-1']].fillna(1)

In [15]:
# df = df.fillna(-1)

#### Case

Skor MAPE sangat sukar untuk stabil karena terdapat beberapa outlier dalam variabel yang ingin diprediksi data. Bahkan perbedaannya tidaklah signifikan dengan model baseline

In [24]:
lgb_params = {'boosting_type': 'gbdt', 
              'objective': 'mape',
              'metric': ['mape'], 
              'learning_rate': 0.1,      
              'subsample': 0.9,       
              'subsample_freq': 1,     
              'num_leaves': 255,            
              'min_data_in_leaf': 255, 
              'feature_fraction': 0.5,
              'n_estimators': 10000,   
              'early_stopping_rounds': 100,
              'seed': SEED,
              'verbose': -1}
for fold in range(FOLD):
    mdl, val_df = process_train_lgb(
        df,
        remove_additional_features=rs_features,
        kfold=fold
    )
val_df.describe()

['kddati2', 'tkp', 'peserta', 'id', 'month', 'year', 'case2', 'unit_cost2', 'peserta_lag_1', 'case_lag_1', 'unit_cost_lag_1', 'case2_lag_1', 'unit_cost2_lag_1', 'peserta_lag_-1', 'case_lag_-1', 'unit_cost_lag_-1', 'case2_lag_-1', 'unit_cost2_lag_-1', 'ds_case2', 'ds_case_lag_1', 'ds_case2_lag_1', 'ds_case_lag_-1', 'ds_case2_lag_-1']
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[34]	training's mape: 0.156895	valid_1's mape: 0.196103
MAPE CV-1 is 19.61%
MAE CV-1 is 1057.83
['kddati2', 'tkp', 'peserta', 'id', 'month', 'year', 'case2', 'unit_cost2', 'peserta_lag_1', 'case_lag_1', 'unit_cost_lag_1', 'case2_lag_1', 'unit_cost2_lag_1', 'peserta_lag_-1', 'case_lag_-1', 'unit_cost_lag_-1', 'case2_lag_-1', 'unit_cost2_lag_-1', 'ds_case2', 'ds_case_lag_1', 'ds_case2_lag_1', 'ds_case_lag_-1', 'ds_case2_lag_-1']
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[135]	training's mape: 0.11945	valid_1's

,row_id,case,predict_case,kfold
count,11593.000000,11593.000000,11593.000000,11593.0
mean,28897.505305,6479.784266,5935.962227,4.0
std,16640.971428,17332.282285,14659.078794,0.0
min,2.000000,1.000000,1.609590,4.0
25%,14719.000000,425.000000,387.599036,4.0
50%,28825.000000,1346.000000,1261.113902,4.0
75%,43091.000000,4552.000000,4228.508152,4.0
max,57969.000000,288547.000000,131770.318715,4.0


In [25]:
lgb_params = {'boosting_type': 'gbdt', 
              'objective': 'mape',
              'metric': ['mape'], 
              'learning_rate': 0.1,      
              'subsample': 0.9,       
              'subsample_freq': 1,     
              'num_leaves': 255,            
              'min_data_in_leaf': 255, 
              'feature_fraction': 0.9,
              'n_estimators': 10000,   
              'early_stopping_rounds': 100,
              'seed': SEED,
              'verbose': -1}
for fold in range(FOLD):
    mdl, val_df = process_train_lgb(
        df,
        remove_additional_features=rs_features,
        kfold=fold
    )
val_df.describe()

['kddati2', 'tkp', 'peserta', 'id', 'month', 'year', 'case2', 'unit_cost2', 'peserta_lag_1', 'case_lag_1', 'unit_cost_lag_1', 'case2_lag_1', 'unit_cost2_lag_1', 'peserta_lag_-1', 'case_lag_-1', 'unit_cost_lag_-1', 'case2_lag_-1', 'unit_cost2_lag_-1', 'ds_case2', 'ds_case_lag_1', 'ds_case2_lag_1', 'ds_case_lag_-1', 'ds_case2_lag_-1']
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[44]	training's mape: 0.138716	valid_1's mape: 0.167775
MAPE CV-1 is 16.78%
MAE CV-1 is 978.82
['kddati2', 'tkp', 'peserta', 'id', 'month', 'year', 'case2', 'unit_cost2', 'peserta_lag_1', 'case_lag_1', 'unit_cost_lag_1', 'case2_lag_1', 'unit_cost2_lag_1', 'peserta_lag_-1', 'case_lag_-1', 'unit_cost_lag_-1', 'case2_lag_-1', 'unit_cost2_lag_-1', 'ds_case2', 'ds_case_lag_1', 'ds_case2_lag_1', 'ds_case_lag_-1', 'ds_case2_lag_-1']
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[42]	training's mape: 0.140259	valid_1's 

,row_id,case,predict_case,kfold
count,11593.000000,11593.000000,11593.000000,11593.0
mean,28897.505305,6479.784266,5834.003850,4.0
std,16640.971428,17332.282285,14173.471503,0.0
min,2.000000,1.000000,1.000000,4.0
25%,14719.000000,425.000000,387.684680,4.0
50%,28825.000000,1346.000000,1230.049359,4.0
75%,43091.000000,4552.000000,4255.115386,4.0
max,57969.000000,288547.000000,130359.397868,4.0


In [28]:
df2 = df.fillna(-1)
lgb_params = {'boosting_type': 'gbdt', 
              'objective': 'mape',
              'metric': ['mape'], 
              'learning_rate': 0.1,      
              'subsample': 0.9,       
              'subsample_freq': 1,     
              'num_leaves': 255,            
              'min_data_in_leaf': 255, 
              'feature_fraction': 0.9,
              'n_estimators': 10000,   
              'early_stopping_rounds': 100,
              'seed': SEED,
              'verbose': -1}
for fold in range(FOLD):
    mdl, val_df = process_train_lgb(
        df2,
        remove_additional_features=rs_features,
        kfold=fold
    )
val_df.describe()

['kddati2', 'tkp', 'peserta', 'id', 'month', 'year', 'case2', 'unit_cost2', 'peserta_lag_1', 'case_lag_1', 'unit_cost_lag_1', 'case2_lag_1', 'unit_cost2_lag_1', 'peserta_lag_-1', 'case_lag_-1', 'unit_cost_lag_-1', 'case2_lag_-1', 'unit_cost2_lag_-1', 'ds_case2', 'ds_case_lag_1', 'ds_case2_lag_1', 'ds_case_lag_-1', 'ds_case2_lag_-1']
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[42]	training's mape: 0.140589	valid_1's mape: 0.166972
MAPE CV-1 is 16.70%
MAE CV-1 is 999.41
['kddati2', 'tkp', 'peserta', 'id', 'month', 'year', 'case2', 'unit_cost2', 'peserta_lag_1', 'case_lag_1', 'unit_cost_lag_1', 'case2_lag_1', 'unit_cost2_lag_1', 'peserta_lag_-1', 'case_lag_-1', 'unit_cost_lag_-1', 'case2_lag_-1', 'unit_cost2_lag_-1', 'ds_case2', 'ds_case_lag_1', 'ds_case2_lag_1', 'ds_case_lag_-1', 'ds_case2_lag_-1']
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[36]	training's mape: 0.145563	valid_1's 

,row_id,case,predict_case,kfold
count,11593.000000,11593.000000,11593.000000,11593.0
mean,28897.505305,6479.784266,5835.164333,4.0
std,16640.971428,17332.282285,14216.865205,0.0
min,2.000000,1.000000,1.000000,4.0
25%,14719.000000,425.000000,387.852486,4.0
50%,28825.000000,1346.000000,1232.294633,4.0
75%,43091.000000,4552.000000,4236.806625,4.0
max,57969.000000,288547.000000,128733.337197,4.0


In [29]:
df2['rs_total'] = df2[rs_features].sum(axis=1)
lgb_params = {'boosting_type': 'gbdt', 
              'objective': 'mape',
              'metric': ['mape'], 
              'learning_rate': 0.1,      
              'subsample': 0.9,       
              'subsample_freq': 1,     
              'num_leaves': 255,            
              'min_data_in_leaf': 255, 
              'feature_fraction': 0.9,
              'n_estimators': 10000,   
              'early_stopping_rounds': 100,
              'seed': SEED,
              'verbose': -1}
for fold in range(FOLD):
    mdl, val_df = process_train_lgb(
        df2,
        remove_additional_features=rs_features,
        kfold=fold
    )
val_df.describe()

['kddati2', 'tkp', 'peserta', 'id', 'month', 'year', 'case2', 'unit_cost2', 'peserta_lag_1', 'case_lag_1', 'unit_cost_lag_1', 'case2_lag_1', 'unit_cost2_lag_1', 'peserta_lag_-1', 'case_lag_-1', 'unit_cost_lag_-1', 'case2_lag_-1', 'unit_cost2_lag_-1', 'ds_case2', 'ds_case_lag_1', 'ds_case2_lag_1', 'ds_case_lag_-1', 'ds_case2_lag_-1', 'rs_total']
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[53]	training's mape: 0.133921	valid_1's mape: 0.163086
MAPE CV-1 is 16.31%
MAE CV-1 is 965.92
['kddati2', 'tkp', 'peserta', 'id', 'month', 'year', 'case2', 'unit_cost2', 'peserta_lag_1', 'case_lag_1', 'unit_cost_lag_1', 'case2_lag_1', 'unit_cost2_lag_1', 'peserta_lag_-1', 'case_lag_-1', 'unit_cost_lag_-1', 'case2_lag_-1', 'unit_cost2_lag_-1', 'ds_case2', 'ds_case_lag_1', 'ds_case2_lag_1', 'ds_case_lag_-1', 'ds_case2_lag_-1', 'rs_total']
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[37]	training's m

,row_id,case,predict_case,kfold
count,11593.000000,11593.000000,11593.000000,11593.0
mean,28897.505305,6479.784266,5769.604320,4.0
std,16640.971428,17332.282285,13830.515146,0.0
min,2.000000,1.000000,1.000000,4.0
25%,14719.000000,425.000000,384.169893,4.0
50%,28825.000000,1346.000000,1245.550997,4.0
75%,43091.000000,4552.000000,4201.195267,4.0
max,57969.000000,288547.000000,123057.260455,4.0


In [26]:
lgb_params = {'boosting_type': 'gbdt', 
              'objective': 'mape',
              'metric': ['mape'], 
              'learning_rate': 0.1,      
              'subsample': 0.9,       
              'subsample_freq': 1,     
              'num_leaves': 255,            
              'min_data_in_leaf': 255, 
              'feature_fraction': 0.5,
              'n_estimators': 10000,   
            #   'early_stopping_rounds': 100,
              'seed': SEED,
              'verbose': -1}
for fold in range(FOLD):
    mdl, val_df = process_train_lgb(
        df,
        remove_additional_features=rs_features,
        kfold=fold
    )
val_df.describe()

['kddati2', 'tkp', 'peserta', 'id', 'month', 'year', 'case2', 'unit_cost2', 'peserta_lag_1', 'case_lag_1', 'unit_cost_lag_1', 'case2_lag_1', 'unit_cost2_lag_1', 'peserta_lag_-1', 'case_lag_-1', 'unit_cost_lag_-1', 'case2_lag_-1', 'unit_cost2_lag_-1', 'ds_case2', 'ds_case_lag_1', 'ds_case2_lag_1', 'ds_case_lag_-1', 'ds_case2_lag_-1']
[500]	training's mape: 0.0992367	valid_1's mape: 0.328336
[1000]	training's mape: 0.0914793	valid_1's mape: 0.350258
[1500]	training's mape: 0.0848183	valid_1's mape: 0.349169
[2000]	training's mape: 0.0821405	valid_1's mape: 0.356159
[2500]	training's mape: 0.0790653	valid_1's mape: 0.354836
[3000]	training's mape: 0.0780972	valid_1's mape: 0.35442
[3500]	training's mape: 0.0769483	valid_1's mape: 0.357582
[4000]	training's mape: 0.0750328	valid_1's mape: 0.35328
[4500]	training's mape: 0.0742562	valid_1's mape: 0.353806
[5000]	training's mape: 0.0729149	valid_1's mape: 0.354696
[5500]	training's mape: 0.0719126	valid_1's mape: 0.353293
[6000]	training's m

KeyboardInterrupt: 

#### Cost

Tidak jauh lebih baik dari baseline

In [25]:
lgb_params = {'boosting_type': 'gbdt', 
              'objective': 'mape',
              'metric': ['mape'], 
              'learning_rate': 0.2,      
              'subsample': 0.9,       
              'subsample_freq': 1,     
              'num_leaves': 255,            
              'min_data_in_leaf': 255, 
              'feature_fraction': 0.95,
              'n_estimators': 1000,   
              'early_stopping_rounds': 100,
              'seed': SEED,
              'verbose': -1}
for fold in range(FOLD):
    mdl, val_df = process_train_lgb(
        df2[['kddati2','tkp','unit_cost_lag_1','unit_cost_lag_-1','cat','id','unit_cost','row_id','tglpelayanan','kfold']],
        remove_additional_features=rs_features,
        kfold=fold,
        target=TARGET[1]
    )

['kddati2', 'tkp', 'unit_cost_lag_1', 'unit_cost_lag_-1', 'id']
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[76]	training's mape: 0.0432648	valid_1's mape: 0.0428055
MAPE CV-1 is 4.28%
MAE CV-1 is 86278.22
['kddati2', 'tkp', 'unit_cost_lag_1', 'unit_cost_lag_-1', 'id']
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[40]	training's mape: 0.0427262	valid_1's mape: 0.0493831
MAPE CV-2 is 4.94%
MAE CV-2 is 86490.30
['kddati2', 'tkp', 'unit_cost_lag_1', 'unit_cost_lag_-1', 'id']
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[86]	training's mape: 0.0421798	valid_1's mape: 0.0453531
MAPE CV-3 is 4.54%
MAE CV-3 is 87303.31
['kddati2', 'tkp', 'unit_cost_lag_1', 'unit_cost_lag_-1', 'id']
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[80]	training's mape: 0.0415888	valid_1's mape: 0.0470467
MAPE